In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # Colab 환경설정

# !pip install gluonnlp pandas tqdm
# !pip install mxnet
# !pip install sentencepiece==0.1.91
# !pip install transformers==4.8.2
# !pip install torch==1.8.1

In [ ]:
!nvidia-smi

In [3]:
!kill -9 17172

In [ ]:
# github에서 Kobert 파일 로드 및 Kobert 모델 불러오기

# #kobert_tokenizer 폴더 다운받는 코드
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

# #https://github.com/SKTBrain/KoBERT 의 파일들을 다운받는 코드
# !pip install 'git+https://git@github.com/SKTBrain/KoBERT.git@master'

from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_pytorch_kobert_model()

In [2]:
# 필요한 라이브러리 불러오기

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

In [3]:
# 입력 데이터셋을 토큰화하기

# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding padding 등을 해주는 코드이다.
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [4]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [5]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [63]:
## 학습 모델 로드
PATH = './Model/'
model = torch.load(PATH + 'Model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
# model.load_state_dict(torch.load(PATH + 'epoch5/Model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

In [64]:
#토큰화
tok=tokenizer.tokenize

In [65]:
df_test = pd.read_csv("./dataset/test.csv", encoding="utf-8")
df_test

,발화문,상황,1번 감정,2번 감정,3번 감정,4번 감정,5번 감정,Emotion
0,별건 아니고 이번에 마라톤 대회에서 내 기록이 많이 좋아졌어.,happiness,neutral,happiness,happiness,happiness,surprise,happiness
1,그런 놈은 욕을 쳐먹을려고 하는 건데 욕을 해줘야될거 아니야.,angry,angry,angry,angry,angry,angry,angry
2,코로나 때문에 나가지도 못하고 답답해.,sadness,sadness,sadness,neutral,happiness,neutral,sadness
3,너의 말대로 사진을 찍어두는게 좋을 것 같아!,disgust,disgust,angry,sadness,disgust,sadness,disgust
4,응. 날씨는 좋았어.,neutral,sadness,neutral,neutral,neutral,neutral,neutral
...,...,...,...,...,...,...,...,...
1861,"어, 다행히 아무 일도 없었어.",surprise,neutral,surprise,sadness,neutral,neutral,neutral
1862,그냥 비대면 수업이라 학교 안 가서 좋아. 할만 해.,neutral,angry,fear,sadness,neutral,neutral,neutral
1863,당연하지. 한 달간 얼마나 고생했는데!,happiness,neutral,happiness,sadness,happiness,angry,happiness
1864,화장실은 청소를 해도 곰팡이가 없어지질 않아.,angry,disgust,angry,disgust,disgust,angry,disgust


In [66]:
# # 단발성대화 데이터셋 변형
# df_test = df_test.replace({"Emotion":{"중립": 0, "행복": 1, "슬픔": 2, "분노": 3, "혐오": 4, "공포": 5, "놀람": 6}})

In [67]:
df_test = df_test.replace({"Emotion":{"neutral": 0, "happiness": 1, "sadness": 2, "angry": 3, "disgust": 4, "fear": 5, "surprise": 6}})

In [68]:
df_test

,발화문,상황,1번 감정,2번 감정,3번 감정,4번 감정,5번 감정,Emotion
0,별건 아니고 이번에 마라톤 대회에서 내 기록이 많이 좋아졌어.,happiness,neutral,happiness,happiness,happiness,surprise,1
1,그런 놈은 욕을 쳐먹을려고 하는 건데 욕을 해줘야될거 아니야.,angry,angry,angry,angry,angry,angry,3
2,코로나 때문에 나가지도 못하고 답답해.,sadness,sadness,sadness,neutral,happiness,neutral,2
3,너의 말대로 사진을 찍어두는게 좋을 것 같아!,disgust,disgust,angry,sadness,disgust,sadness,4
4,응. 날씨는 좋았어.,neutral,sadness,neutral,neutral,neutral,neutral,0
...,...,...,...,...,...,...,...,...
1861,"어, 다행히 아무 일도 없었어.",surprise,neutral,surprise,sadness,neutral,neutral,0
1862,그냥 비대면 수업이라 학교 안 가서 좋아. 할만 해.,neutral,angry,fear,sadness,neutral,neutral,0
1863,당연하지. 한 달간 얼마나 고생했는데!,happiness,neutral,happiness,sadness,happiness,angry,1
1864,화장실은 청소를 해도 곰팡이가 없어지질 않아.,angry,disgust,angry,disgust,disgust,angry,4


In [69]:
def predict(df_test):
    data_test = df_test['발화문'].tolist()
    
    data = []
    for i in range(len(data_test)):
        data.append([data_test[i], 0])
        
    testdataset = BERTDataset(data, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(testdataset, batch_size=batch_size, num_workers=5)
    
    model.eval()

    label_list = []
#     confidence_list = []
    for token_ids, valid_length, segment_ids, label in tqdm(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        out = out.detach().cpu()
        logits = nn.Softmax().forward(out)
        value, idx = logits.max(dim=-1)

        label_list.extend(idx.tolist())
    
    return label_list
#         confidence_list += value.tolist()

In [70]:
# 오차행렬, 정확도, 정밀도, 재현율 f1 score을 한번에 출력하기 위한 함수 정의

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_eval(y_true, pred):
#     confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_true, pred)
    precision = precision_score(y_true, pred, average= "weighted")
    recall = recall_score(y_true, pred, average= "weighted")
    f1 = f1_score(y_true, pred, average= "weighted")
#     print('Confusion Matrix')
#     print(confusion)
    print('정확도:{}, 정밀도:{}, 재현율:{} f1 score:{}'.format(accuracy, precision, recall, f1))

In [71]:
y_true = df_test['Emotion'].tolist()

In [72]:
y_pred = predict(df_test)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_32517/1221915933.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = predict(df_test)
100%|██████████| 30/30 [00:02<00:00, 11.66it/s]


In [73]:
get_clf_eval(y_true, y_pred)

정확도:0.46355841371918544, 정밀도:0.5324069898045083, 재현율:0.4985921855637998 f1 score:0.5072071717322884


In [74]:
# 예측 모델 설정
def predict_unit(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("중립이")
            elif np.argmax(logits) == 1:
                test_eval.append("행복이")
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 3:
                test_eval.append("분노가")
            elif np.argmax(logits) == 4:
                test_eval.append("혐오가")
            elif np.argmax(logits) == 5:
                test_eval.append("공포가")
            elif np.argmax(logits) == 6:
                test_eval.append("놀람이")
          

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict_unit(sentence)
    print("\n")